In [1]:
import scraping_packers_homepage.env as env
llm = env.llm
target_column_list = env.TARGET_COLUMN_LIST
print(target_column_list)

/Users/yoonhae/.pyenv/versions/3.8.12/envs/crawling/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.vertexai.VertexAI` was deprecated in langchain-community 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAI`.
  warn_deprecated(


['homepage', 'extract_url', 'company_name', 'mail', 'phone', 'address', 'fax', 'products', 'error']


In [2]:
import pandas as pd
import scraping_packers_homepage.web_scraper as web_scraper
import scraping_packers_homepage.llm_models.firm_info_extractor as firm_info_extractor
import scraping_packers_homepage.llm_models.a_tag_selector as a_tag_selector
import importlib
# 모듈이 수정된 후에, 다음 코드를 실행하여 모듈을 다시 로드합니다.
importlib.reload(web_scraper)


url_list = [
    #'https://www.naturzgroup.com/',  # 60
    #'https://kayalfoods.in/',  # 4556
    #"http://www.great-lines.com/about-us/"
    #"http://bonanzamegahoil.com"
    #"https://www.classicpaints.com.fj"
    "https://www.exportersindia.com/kube-exim/"
    #"https://www.escurette.fr/"
    #'https://www.torresyribelles.com/home2'
]

#result_df = pd.DataFrame(columns=target_column_list, dtype=str)
for url in url_list:
    sub_df = web_scraper.run(url, llm, target_column_list, required_column_list=["mail", "phone", "products"])
    #result_df = result_df.append(sub_df, ignore_index=True)

#len(result_df)

** [main page] 조회 - https://www.classicpaints.com.fj
  ** [llm] contact 정보 & products 추출
    ** split text - chunk : 2
    {
        "contact": {
            "email": null,
            "phone": null,
            "address": null
        },
        "products": [
            "APCO",
            "CarPro",
            "Cromax",
            "Dulux",
            "Easicoat",
            "Meguiars",
            "PPG",
            "Syrox",
            "3M"
        ]
    }
    {
        "contact": {
            "email": null,
            "phone": null,
            "address": null
        },
        "products": [
            "APCO",
            "MEGUIAR'S",
            "CROMAX",
            "CARPRO",
            "DULUX",
            "3M",
            "SYROX"
        ]
    }
  ** a tag 추출
  ** [llm] 회사 정보와 관련있는 a tag 선별
    ['https://www.classicpaints.com.fj/products/']


In [ ]:
def unique_list(series):
    # NaN 값을 제거하고, 중복을 제거한 후 리스트로 변환
    return series.dropna().drop_duplicates().tolist()


# 'homepage'를 기준으로 그룹화하고 각 컬럼에 unique_list 함수 적용
grouped_df = result_df.groupby('homepage').agg(unique_list).reset_index()

def convert_to_string(value):
    # 값이 리스트인 경우
    if isinstance(value, list):
        # 빈 리스트인 경우 None 반환main
        if not value:
            return ''
        # 리스트의 요소들을 문자열로 결합
        return ";\n".join(str(item) for item in value if pd.notna(item) and isinstance(item, str) and item)
    # 리스트가 아닌 경우, 값을 그대로 반환
    return value

# 'homepage' 컬럼을 제외한 모든 컬럼에 대해 함수 적용
for col in grouped_df.columns:
    if col != 'homepage':
        grouped_df[col] = grouped_df[col].apply(convert_to_string)

print(grouped_df)


In [ ]:
print(grouped_df["products"][0])

In [ ]:
import csv
grouped_df.to_csv('./output/check.csv', quoting=csv.QUOTE_ALL, index=False)

# 작업 방법 노트!!!!

1. 컬럼은 "이메일 작업 대상" 이 있는곳까지만 있는지 체크.
2. 작업대상 컬럼들은 진짜 다음 스텝에서 작업해야하는 경우에만 체크해야한다.
3. 홈페이지 크롤링 통해서 추가 데이터가 수집된 경우 작업대상을 그때그때 마킹한다.
4. 작업 마무리 되면 gcs/.../raw  폴더에 drop

## 고민 포인트
> => homepage 크롤링의 작업 소요 시간이 길다. -> 분산처리 필요.
>    => 분산처리를 어떻게 구성할 것인가?
> => 한번 작업한 homepage 에 대한 상세 내역을 따로 저장해둘것인가?
>    => 작업한 이력이 확인되면 저장된 값을 그대로 갖다 쓰는 방향으로?

## 고민 끝나고 작업하기로! 일단 ai 챗봇하러 간당... 20240130

In [10]:
import pandas as pd
new_df = pd.read_csv('/Users/yoonhae/Downloads/(그레인스캐너-데이터팀) 패커 크롤링 - paris_show_재작업.csv')

job_config = {
    "homepage_column": '홈페이지 주소',
    "target_column_dict": {
        "address": {
            "read_col_list": ['국가(영어로)', '주소 raw'],
            "write_col": '주소 raw',
        },
        "products": {
            "read_col_list": ['SKU raw', '취급 SKU1'],
            "write_col": 'SKU raw',
        },
        "phone": {
            "read_col_list": ['전화번호', '기존 전화번호'],
            "write_col": '기존 전화번호',
        },
        "mail": {
            "read_col_list": ['대표 이메일'],
            "write_col": '대표 이메일',
        }
    }
}



def unique_list(series):
    # NaN 값을 제거하고, 중복을 제거한 후 리스트로 변환
    return series.dropna().drop_duplicates().tolist()


def convert_to_string(value):
    # 값이 리스트인 경우
    if isinstance(value, list):
        # 빈 리스트인 경우 None 반환
        if not value:
            return ''
        # 리스트의 요소들을 문자열로 결합
        return ";\n".join(str(item) for item in value if pd.notna(item) and isinstance(item, str) and item)
    # 리스트가 아닌 경우, 값을 그대로 반환
    return value

In [7]:

import scraping_packers_homepage.web_scraper as web_scraper
import scraping_packers_homepage.llm_models.a_tag_selector as a_tag_selector
import scraping_packers_homepage.llm_models.firm_info_extractor as firm_info_extractor
import importlib
# 모듈이 수정된 후에, 다음 코드를 실행하여 모듈을 다시 로드합니다.

#importlib.reload(a_tag_selector)
importlib.reload(web_scraper)
#importlib.reload(firm_info_extractor)

<module 'scraping_packers_homepage.web_scraper' from '/Users/yoonhae/test/crawling/scraping_packers_homepage/web_scraper.py'>

In [ ]:
new_df['패커명*'][347]

In [ ]:

import scraping_packers_homepage.web_scraper as web_scraper
homepage_col = job_config['homepage_column']
target_column_dict = job_config['target_column_dict']

for index, row in new_df.iterrows():
    # if index < 348:
    #     continue
    # 홈페이지 주소가 있는 경우
    if not pd.isna(row[homepage_col]):
        required_column_list = []
        # 수집할 컬럼리스트 추출
        for check_col, job_info in target_column_dict.items():
            if any(not pd.isna(row[col]) for col in job_info['read_col_list'] if col in row):
                # 이미 있어서 구지 수집할 필요없음.
                pass
            else:
                required_column_list.append(check_col)
        # print(row.tolist())
        # print(row[homepage_col], required_column_list)

        # 필요한 데이터가 있는 경우
        if required_column_list and 'http://www.ladrome.fr/' != row[homepage_col]:
            # 스크래핑 작업 진행.
            sub_df = web_scraper.run(row[homepage_col], llm, target_column_list, required_column_list=required_column_list)
            # 사이트별 하나로 그루핑
            sub_df = sub_df.groupby('homepage').agg(unique_list).reset_index()

            for col in required_column_list:
                # 값이 있는 경우
                if sub_df[col].notna().any() and not sub_df[col].empty:
                    # 그루핑 값을 string 기반으로 변경
                    sub_df[col] = sub_df[col].apply(convert_to_string)
                    # 저장
                    new_df.at[index, target_column_dict[col]['write_col']] = sub_df[col].iloc[0]

    if (index % 10) == 9:
        new_df.to_csv('./output/salon_agriculture_20240206_1222.csv', index=False, quoting=1)

new_df.to_csv('./output/salon_agriculture_20240206_1222.csv', index=False, quoting=1)              

In [ ]:
import concurrent.futures
import time

# 실행하고자 하는 함수
def my_function():
    # 여기에 로직 구현
    time.sleep(30)
    pass

# 타임아웃 시간 (초 단위)
timeout_seconds = 5

# ThreadPoolExecutor를 사용하여 별도의 스레드에서 함수 실행
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    # Future 객체를 사용하여 함수 실행
    future = executor.submit(my_function)
    try:
        # 지정된 타임아웃 시간 내에 함수의 실행 결과를 기다림
        result = future.result(timeout=timeout_seconds)
    except concurrent.futures.TimeoutError:
        print("타임아웃: 함수 실행 시간이 초과되었습니다.")
        # 필요한 예외 처리 또는 로직을 여기에 구현


In [ ]:
new_df = pd.read_csv('output/크롤링 데이터 정리_cotton - GMP+ FRA,FSA 1차정리_from_homepage_20240129.csv')


In [ ]:
if "new_contry" in new_df.columns:
    new_df["국가(영어로)"] = new_df["new_contry"]

In [ ]:
import csv
filtered_df = new_df.iloc[:, :new_df.columns.get_loc('이메일 작업 대상') + 1]
filtered_df.to_csv("./output/크롤링 데이터 정리_cotton - GMP+ FRA,FSA 1차정리_from_homepage_20240130.csv", quoting=csv.QUOTE_ALL, index=False)

In [ ]:

import numpy as np    

new_df["주소 작업대상"] = ''
new_df['SKU 작업대상'] = ''
new_df['이메일 작업 대상'] = ''

new_df['주소 작업대상'] = np.where((new_df['주소 raw'].notnull() & new_df['주소 raw'].notna() & (new_df['주소 raw'] != '')) &
                   (new_df['국가(영어로)'].isnull() | new_df['국가(영어로)'].isna() | (new_df['국가(영어로)'] == '')),
                   '작업대상', '')

new_df['SKU 작업대상'] = np.where((new_df['SKU raw'].notnull() & new_df['SKU raw'].notna() & (new_df['SKU raw'] != '')) &
                   (new_df['취급 SKU1'].isnull() | new_df['취급 SKU1'].isna() | (new_df['취급 SKU1'] == '')),
                   '작업대상', '')
if (new_df['참조 이메일(여러개 입력시 콤마(,)로 구분)'].notna() & (new_df['참조 이메일(여러개 입력시 콤마(,)로 구분)'] != '')).any() and \
   ((new_df['대표 이메일'].str.count('@') > 1) | 
    (new_df['대표 이메일'].str.contains('[ ,;]'))).any():
    new_df['이메일 작업 대상'] = np.where((new_df['대표 이메일'].notnull() & new_df['대표 이메일'].notna() & (new_df['대표 이메일'] != '')),
                                '작업대상', '')
    
filtered_df = new_df.iloc[:, :new_df.columns.get_loc('이메일 작업 대상') + 1]
filtered_df.to_csv("./output/크롤링 데이터 정리_cotton - GMP+ FRA,FSA 1차정리_from_homepage_20240130.csv", quoting=csv.QUOTE_ALL, index=False)

In [ ]:
from requests.adapters import HTTPAdapter, Retry
import requests

session = requests.Session()
retries = Retry(total=2, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount('http://', HTTPAdapter(max_retries=retries))
session.timeout = 5

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}
response = session.get("http://www.ladrome.fr/", timeout=5, headers=headers)

In [ ]:
response